In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
import joblib
import random

SEED = 42
random.seed(SEED)           # Python random seed
np.random.seed(SEED)        # NumPy random seed
torch.manual_seed(SEED)     # PyTorch CPU random seed

# 1. load database

In [2]:
excel_file = pd.ExcelFile('SciGlass_Plus_properties.xlsx')
df = excel_file.parse('Sheet1')

In [3]:
comp_list = ['SiO2','P2O5','ZrO2','Na2O','Al2O3','Fe2O3','CaO','MgO','K2O','MnO','GeO2','Li2O','Ta2O5','ZnO','SrO','CdO','SnO2','B2O3','La2O3','Ga2O3','Y2O3','TiO2','Nb2O5','PbO','HfO2','WO3','Sb2O3','Bi2O3','BaO','Cr2O3','Cu2O','BeO','CuO','Nd2O3','CeO2','Cs2O','As2O3','Rb2O','MoO3','FeO','Mn2O3','ThO2','Ag2O','TeO2','Tl2O','CoO','In2O3','Sc2O3','NiO','V2O5','As2O5','MnO2','Sm2O3','Gd2O3','Tb2O3','Dy2O3','Ho2O3','Er2O3','Yb2O3','Co3O4','Fe3O4','SnO','Mn3O4','Ce2O3','Pr2O3','CrO3','VO6','TeO3','UO2','Sb2O5','Pr6O11','VO2','Co2O3','Ti2O3','UO3','Eu2O3','Mo2O3','Ni2O3','MoO','PrO2','TbO2','V2O3','SeO2','Lu2O3','GeO','SbO2','U3O8','Mn2O7','HgO','Tm2O3','Nb2O3','PbO2','Tl2O3','Pb3O4','SiO','Sn2O3','Ta2O3','RuO2','Tb4O7','Tb3O7','SeO3','Cr3O4','PdO','MoO2','Rh2O3','TiO','RhO2','ErO2','YbO2','Bi2O5','EuO','Pu2O3','CeO','CrO','BaO2','Au2O3','ReO3','Re2O7']

filtered_df = df[(df["YoungModulus"] > 0) & (df["AnnealTemperature"] > 0) & (df[comp_list].sum(axis=1) > 0)]
filtered_df_unique = filtered_df.drop_duplicates(subset=comp_list + ["AnnealTemperature"])

In [4]:
y = filtered_df_unique["YoungModulus"].values.reshape(-1, 1)

In [5]:
y.shape

(747, 1)

# 2. pre-processing

In [6]:
def filter_bad_compositions(df, comp_list, tol=1.0, normalize=False):
    
    df = df.copy()
    df["CompSum"] = df[comp_list].sum(axis=1)
    
    lower, upper = 100 - tol, 100 + tol
    
    df_clean = df[(df["CompSum"] >= lower) & (df["CompSum"] <= upper)].copy()
    removed = len(df) - len(df_clean)
    
    if normalize:
        df_clean[comp_list] = df_clean[comp_list].div(df_clean["CompSum"], axis=0) * 100
    
    df_clean = df_clean.drop(columns=["CompSum"])
    return df_clean

def clean_near_duplicates(df, feature_cols, target_col="YoungModulus", 
                          x_tol=0.05, y_tol=0.3):

    # -------- Step 1. 去掉完全重复的 X --------
    #df_unique = df.groupby(feature_cols, as_index=False).agg({target_col: "mean"})
    df_unique = df

    X = df_unique[feature_cols].fillna(0).values
    y = df_unique[target_col].fillna(0).values
    
    dist_matrix = pairwise_distances(X, metric="euclidean")
    np.fill_diagonal(dist_matrix, np.inf)
    
    bad_idx = set()
    for i in range(len(X)):

        close_idx = np.where(dist_matrix[i] < x_tol)[0]
        for j in close_idx:
            if abs(y[i] - y[j]) > y_tol * np.mean([y[i], y[j]]):
                bad_idx.add(i)
                bad_idx.add(j)
    
    df_clean = df_unique.drop(df_unique.index[list(bad_idx)]).reset_index(drop=True)
    return df_clean

filtered_df_clean0 = filter_bad_compositions(
    filtered_df_unique,
    comp_list=comp_list,
    tol=1.0,         
    normalize=False
)

filtered_df_clean = clean_near_duplicates(
    filtered_df_clean0,
    feature_cols = comp_list + ["AnnealTemperature"], 
    target_col = "YoungModulus",
    x_tol = 0.5,
    y_tol = 0.2
)

X1 = filtered_df_clean[comp_list].fillna(0).values/ 100.0
y = filtered_df_clean["YoungModulus"].values.reshape(-1, 1)

X2 = filtered_df_clean[comp_list + ["AnnealTemperature"]].fillna(0).copy()
X2.loc[:, comp_list] = X2[comp_list] / 100.0

In [7]:
filtered_df_clean.shape

(563, 728)

# 3. train, valid, and test

In [8]:
scaler_temp = StandardScaler().fit(X2[["AnnealTemperature"]])
X2.loc[:, "AnnealTemperature"] = scaler_temp.transform(X2[["AnnealTemperature"]])

scaler_X1 = StandardScaler().fit(X1)
scaler_X2 = StandardScaler().fit(X2)
scaler_y = StandardScaler().fit(y)

X1_scaled = X1
X2_scaled = X2.values
y_scaled = scaler_y.transform(y)

joblib.dump(scaler_X1, "scaler_X1.pkl")
joblib.dump(scaler_X2, "scaler_X2.pkl")
joblib.dump(scaler_y, "scaler_y.pkl")

def create_dataloaders(X, y, batch_size=64):
    indices = np.arange(len(X))
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32),
                            torch.tensor(y, dtype=torch.float32),
                            torch.tensor(indices, dtype=torch.long))
    
    n_total = len(dataset)
    n_train = int(0.7 * n_total)
    n_val = int(0.15 * n_total)
    n_test = n_total - n_train - n_val
    g = torch.Generator().manual_seed(SEED)
    train_ds, val_ds, test_ds = random_split(dataset, [n_train, n_val, n_test], generator=g)
    
    return (DataLoader(train_ds, batch_size=batch_size, shuffle=False),
            DataLoader(val_ds, batch_size=batch_size),
            DataLoader(test_ds, batch_size=batch_size))

train1, val1, test1 = create_dataloaders(X1_scaled, y)
train2, val2, test2 = create_dataloaders(X2_scaled, y)


class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x)


def train_model(model, train_dl, val_dl, epochs=1000, lr=1e-3):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    val_losses = []

    for epoch in range(epochs):
        model.train()
        for xb, yb, _ in train_dl:
            pred = model(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb, yb, _ in val_dl:
                val_loss += criterion(model(xb), yb).item()
        val_loss /= len(val_dl)
        

        if (epoch+1) % 50 == 0:
            print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")
            val_losses.append(val_loss)

    return model, val_losses


def evaluate_and_save_and_analyze(model, dl, X_original, scaler_y, filename_prefix):
    model.eval()
    preds, trues, orig_idx = [], [], []
    with torch.no_grad():
        for xb, yb, idx in dl:
            pred = model(xb)
            preds.append(pred.numpy())
            trues.append(yb.numpy())
            orig_idx.append(idx.numpy())
    
    preds = np.vstack(preds)
    trues = np.vstack(trues)
    orig_idx = np.hstack(orig_idx)
    
    rmse = np.sqrt(np.mean((preds - trues) ** 2))
    
    np.savetxt(f"{filename_prefix}_pred_vs_true.txt", np.hstack([trues, preds]),
               header="true_value\tpredicted_value")
    
    df_results = X_original.iloc[orig_idx].copy()
    df_results['True'] = trues
    df_results['Pred'] = preds
    df_results['Error'] = df_results['Pred'] - df_results['True']
    
    return rmse, df_results


def train_and_evaluate_multiple_runs(model_class, input_dim, dataloaders, X_original, scaler_y, filename_prefix, 
                                     n_runs=10, epochs=1500, lr=1e-3):
    all_preds, all_trues, all_indices = [], [], []
    rmses = []
    all_val_curves = []  

    for run in range(n_runs):
        print(f"\n===== Run {run+1}/{n_runs} =====")
        model = model_class(input_dim)
        model, val_losses = train_model(model, dataloaders[0], dataloaders[1], epochs=epochs, lr=lr)

        torch.save(model.state_dict(), f"{filename_prefix}_run{run+1}.pt")

        rmse, df_results = evaluate_and_save_and_analyze(
            model, dataloaders[2], X_original, scaler_y, f"{filename_prefix}_run{run+1}"
        )
        rmses.append(rmse)

        all_preds.append(df_results['Pred'].values.reshape(-1, 1))
        all_trues.append(df_results['True'].values.reshape(-1, 1))
        all_indices.append(df_results.index.values)
        all_val_curves.append(val_losses)

    
    all_indices = np.array(all_indices)
    assert np.all(all_indices == all_indices[0]), "Indices mismatch across runs!"
    
    
    preds_mean = np.mean(np.hstack(all_preds), axis=1).reshape(-1, 1)
    trues_mean = all_trues[0]
    rmse_mean = np.sqrt(np.mean((preds_mean - trues_mean) ** 2))

    df_mean = X_original.iloc[all_indices[0]].copy()
    df_mean["True"] = trues_mean
    df_mean["Pred_mean"] = preds_mean
    df_mean["Error_mean"] = df_mean["Pred_mean"] - df_mean["True"]
    df_mean.to_csv(f"{filename_prefix}_mean_results.csv", index=False)

    
    all_val_curves = np.array(all_val_curves)   # shape = (n_runs, epochs)
    val_mean = np.mean(all_val_curves, axis=0)
    val_std = np.std(all_val_curves, axis=0)

    df_curve = pd.DataFrame({
        "epoch": np.arange(1, epochs+1, 50),
        "val_loss_mean": val_mean,
        "val_loss_std": val_std
    })
    df_curve.to_csv(f"{filename_prefix}_val_curve.csv", index=False)

    print(f"\n{filename_prefix}: 平均 RMSE = {rmse_mean:.3f}, 各次 RMSE = {rmses}")
    return rmse_mean, df_mean, df_curve



# Model 1: composition only
rmse1_mean, df1_mean, curve1 = train_and_evaluate_multiple_runs(
    MLP, X1_scaled.shape[1], (train1, val1, test1), filtered_df_clean, scaler_y, "model1", n_runs=10
)

# Model 2: composition + AnnealTemperature
rmse2_mean, df2_mean, curve2 = train_and_evaluate_multiple_runs(
    MLP, X2_scaled.shape[1], (train2, val2, test2), filtered_df_clean, scaler_y, "model2", n_runs=10
)


/var/folders/fk/ztcjdp395590k9d3hts6xcbw0000gn/T/ipykernel_1751/638799307.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[[-2.36014983]
 [-2.36014983]
 [ 0.89370338]
 [ 0.89370338]
 [ 0.89370338]
 [ 0.89370338]
 [ 0.89370338]
 [ 0.89370338]
 [-0.77493929]
 [-0.77493929]
 [-0.77493929]
 [-0.77493929]
 [-0.77493929]
 [-1.60926063]
 [-1.60926063]
 [-1.60926063]
 [-1.60926063]
 [-1.60926063]
 [-1.60926063]
 [-0.35777862]
 [-0.35777862]
 [-0.35777862]
 [-0.35777862]
 [-0.35777862]
 [-0.35777862]
 [-0.35777862]
 [-0.35777862]
 [-0.35777862]
 [ 3.31323525]
 [ 3.31323525]
 [ 3.31323525]
 [ 2.92110423]
 [ 3.89726019]
 [-0.27434649]
 [-0.27434649]
 [-0.27434649]
 [-0.27434649]
 [-0.77493929]
 [ 0.89370338]
 [ 0.89370338]
 [ 0.89370338]
 [ 0.89370338]
 [ 0.89370338]
 [-0.77493929]
 [-0.77493929]
 [-0.77493929]
 [-0.77493929]
 [-0.77493929]
 [ 0.05938204]
 [ 0.05938204]
 [ 0.05938204]
 [-0.35777862]
 [-0.35777862]
 [-0.35


===== Run 1/10 =====
Epoch 50, Validation Loss: 799.1247
Epoch 100, Validation Loss: 794.1595
Epoch 150, Validation Loss: 782.9171
Epoch 200, Validation Loss: 766.5204
Epoch 250, Validation Loss: 747.4532
Epoch 300, Validation Loss: 726.6767
Epoch 350, Validation Loss: 706.3056
Epoch 400, Validation Loss: 690.4323
Epoch 450, Validation Loss: 677.5424
Epoch 500, Validation Loss: 667.0900
Epoch 550, Validation Loss: 656.1079
Epoch 600, Validation Loss: 640.6203
Epoch 650, Validation Loss: 622.9776
Epoch 700, Validation Loss: 604.2815
Epoch 750, Validation Loss: 580.6821
Epoch 800, Validation Loss: 554.2822
Epoch 850, Validation Loss: 532.0562
Epoch 900, Validation Loss: 509.5232
Epoch 950, Validation Loss: 485.2414
Epoch 1000, Validation Loss: 465.2838
Epoch 1050, Validation Loss: 451.0365
Epoch 1100, Validation Loss: 445.5175
Epoch 1150, Validation Loss: 439.8187
Epoch 1200, Validation Loss: 430.5922
Epoch 1250, Validation Loss: 420.5833
Epoch 1300, Validation Loss: 414.8788
Epoch 1350

Epoch 300, Validation Loss: 703.0779
Epoch 350, Validation Loss: 681.2695
Epoch 400, Validation Loss: 664.6148
Epoch 450, Validation Loss: 650.8715
Epoch 500, Validation Loss: 639.3171
Epoch 550, Validation Loss: 629.0811
Epoch 600, Validation Loss: 615.1478
Epoch 650, Validation Loss: 600.7480
Epoch 700, Validation Loss: 585.5789
Epoch 750, Validation Loss: 569.5879
Epoch 800, Validation Loss: 551.1318
Epoch 850, Validation Loss: 532.4848
Epoch 900, Validation Loss: 511.8873
Epoch 950, Validation Loss: 494.9460
Epoch 1000, Validation Loss: 481.1874
Epoch 1050, Validation Loss: 466.7105
Epoch 1100, Validation Loss: 452.2461
Epoch 1150, Validation Loss: 437.9510
Epoch 1200, Validation Loss: 425.9205
Epoch 1250, Validation Loss: 416.4245
Epoch 1300, Validation Loss: 403.3960
Epoch 1350, Validation Loss: 390.2080
Epoch 1400, Validation Loss: 374.9007
Epoch 1450, Validation Loss: 360.9898
Epoch 1500, Validation Loss: 346.0764

===== Run 9/10 =====
Epoch 50, Validation Loss: 800.2228
Epoch 

Epoch 400, Validation Loss: 589.9248
Epoch 450, Validation Loss: 548.8285
Epoch 500, Validation Loss: 508.2536
Epoch 550, Validation Loss: 438.0364
Epoch 600, Validation Loss: 358.0137
Epoch 650, Validation Loss: 303.2509
Epoch 700, Validation Loss: 270.6163
Epoch 750, Validation Loss: 250.3989
Epoch 800, Validation Loss: 238.6647
Epoch 850, Validation Loss: 233.5156
Epoch 900, Validation Loss: 231.6807
Epoch 950, Validation Loss: 229.2785
Epoch 1000, Validation Loss: 227.1199
Epoch 1050, Validation Loss: 223.5247
Epoch 1100, Validation Loss: 220.8455
Epoch 1150, Validation Loss: 220.0146
Epoch 1200, Validation Loss: 220.6351
Epoch 1250, Validation Loss: 219.1726
Epoch 1300, Validation Loss: 218.0353
Epoch 1350, Validation Loss: 215.8617
Epoch 1400, Validation Loss: 212.5839
Epoch 1450, Validation Loss: 208.8293
Epoch 1500, Validation Loss: 203.3826

===== Run 6/10 =====
Epoch 50, Validation Loss: 744.3665
Epoch 100, Validation Loss: 734.1789
Epoch 150, Validation Loss: 720.7626
Epoch 